In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
!pip install tensorflow==1.15
!pip install keras==2.2.5
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-38t9kbp2
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-38t9kbp2
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp37-none-any.whl size=101065 sha256=3f6ccd29e45f563b1971e8b1711d09e8f3544474f696d0ae1f4c7c163b4d1ac8
  Stored in directory: /tmp/pip-ephem-wheel-cache-sha58wra/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [26]:
from keras.models import Sequential, Model, Input
from keras.layers import LSTM, Dense, TimeDistributed, Activation, Bidirectional, Masking, Embedding, Dropout, Flatten, concatenate, Conv1D, MaxPool1D, Lambda, Softmax
from keras import backend as K
from keras_contrib.layers import CRF
from keras.utils import plot_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_contrib.losses import  crf_loss
from keras_contrib.metrics import crf_accuracy

In [27]:
from keras.models import load_model
m = load_model("/content/drive/MyDrive/model/model-use-rdr-4500-1-1-1-ver2.h5")

In [28]:
import numpy as np
from numpy import argmax

In [29]:
import re
def map_number_and_punct(word):
    # check hem va ngach
    num_of_seperate=0
    dem=0
    for char in word:
      if not char.isnumeric() and char!="/":
        dem+=1        
      if char=="/":
        num_of_seperate+=1
    if dem==0:
      if len(word)>1 and num_of_seperate==1:
        return u'<ngach>'
      if len(word)>1 and num_of_seperate>1:
        return u'<hem>'

    # if re.match(r"^[0-9]{2}0{3}$", word):
    #     return u'<postcode>'
    
    if word.isnumeric():
        word = u'<number>'
    elif word in [u',', u'<', u'.', u'>', u'/', u'?', u'..', u'...', u'....', u':', u';', u'"', u"'", u'[', u'{', u']',
                  u'}', u'|', u'\\', u'`', u'~', u'!', u'@', u'#', u'$', u'%', u'^', u'&', u'*', u'(', u')', u'-', u'+',
                  u'=']:
        word = u'<punct>'

    # word = word.replace("_"," ")
    
    return word

In [30]:
import numpy as np
import codecs
def read_data(input_file):
    with codecs.open(input_file, 'r', 'utf-8') as f:
        word_list = [] 
        words = []
        for line in f:
            line = line.split()
            if len(line) > 0:
                words.append( map_number_and_punct(line[0].lower()[0:31]) )
            else:
                word_list.append(words)
                words = []
    return word_list

In [31]:
test_word_sentences = read_data('/content/drive/MyDrive/fast_text/test/data_no_tag_pred.txt')
print(len(test_word_sentences))

900


In [32]:
#get char embedding
def read_char_vocab(file_path):
  char_vocab = []
  for line in open(file_path, encoding='utf-8'):
    char_vocab.append(line.splitlines()[0])      
  return char_vocab

def char_to_int(char_vocab_data):
  dic = {}
  index = -1
  for char in char_vocab_data:
    try:
      dic[char]
    except:
      index = index + 1
      dic[char]=index
  return dic
 
def get_char_encode(sentence,max_length_of_a_sentence,max_length_of_a_word):
  char_vocab_data = read_char_vocab("/content/drive/MyDrive/fast_text/VISCII_short.txt")
  dic = char_to_int(char_vocab_data)
  # sentence  words is a sentence | ['i','am','an']
  sentence_encoded = np.zeros([max_length_of_a_sentence,max_length_of_a_word]) # 25*25
  for j in range(len(sentence)):  
    word = sentence[j].lower()
    # integer_encoded = [char_to_int[char] for char in word]
    word_encoded = np.zeros(max_length_of_a_word)
    for k in range(len(word)):
      char = word[k]
      try:
        word_encoded[k]= dic[char]
      except:
        # print("error : " + str(char)+" "+str(word)+" "+str(len(word)))
        word_encoded[k]= dic['[unk]']
    # sentence encoded
    sentence_encoded[j] = word_encoded
  return sentence_encoded

def get_charEmbedd_form_encode(charEnocde):
  LEN_OF_VOCAB = 137
  shape = charEnocde.shape
  char_embedd = np.zeros([shape[0],shape[1],LEN_OF_VOCAB])
  for i in range(shape[0]):
    for j in range(shape[1]):
      char_int = charEnocde[i,j]
      char_int = char_int.astype(np.int64)
      onehot = np.zeros(LEN_OF_VOCAB)
      onehot[char_int] = 1
      char_embedd[i,j,:] = onehot
  return char_embedd

char_embedd = []
for sen_token in test_word_sentences:
  char_en = get_char_encode(sen_token,42,32)
  char_em = get_charEmbedd_form_encode(char_en)
  char_embedd.append(char_em)

char_embedd = np.array(char_embedd)
print(char_embedd.shape)

(900, 42, 32, 137)


In [33]:
tag = np.loadtxt('/content/drive/MyDrive/fast_text/test/tag_embedd.txt')
tag = tag.reshape(int(tag.shape[0]/42), 42, tag.shape[1])
print(tag.shape)

(900, 42, 13)


In [34]:
word = np.loadtxt('/content/drive/MyDrive/fast_text/test/word_embedd.txt')
word = word.reshape(int(word.shape[0]/42), 42, word.shape[1])
print(word.shape)

(900, 42, 300)


In [35]:
y_pred = m.predict([char_embedd, word])
y_true = tag
print(y_pred.shape)
print(y_true.shape)

(900, 42, 13)
(900, 42, 13)


In [36]:
from sklearn.metrics import classification_report, confusion_matrix

In [37]:
y_pred = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1], y_pred.shape[2])
print(y_pred.shape)
y_pred = np.argmax(y_pred,axis=1)
print(y_pred.shape)
y_pred = y_pred.reshape(int(y_pred.shape[0]/42), 42)
print(y_pred.shape)

(37800, 13)
(37800,)
(900, 42)


In [38]:
y_true = y_true.reshape(y_true.shape[0]*y_true.shape[1], y_true.shape[2])
print(y_true.shape)
y_true = np.argmax(y_true,axis=1)
print(y_true.shape)
y_true = y_true.reshape(int(y_true.shape[0]/42), 42)
print(y_true.shape)

(37800, 13)
(37800,)
(900, 42)


In [39]:
from sklearn.metrics import classification_report, confusion_matrix

In [40]:
def arr_tag_in_a_tag_sen(sen_pred, sen_true, tag):
  j=0
  bat_dau=-1
  ket_thuc=-1
  while j < len(sen_pred):
    if sen_true[j]==tag:
      bat_dau=j
      ket_thuc=j
      while sen_true[ket_thuc+1]==1:
        ket_thuc=ket_thuc+1
      ket_thuc=ket_thuc+1
      break
    else:
        j=j+1

  if bat_dau==-1 and ket_thuc==-1:
    return 0
  
  a = sen_pred[bat_dau:ket_thuc]
  b = sen_true[bat_dau:ket_thuc]
  a = np.array(a)
  b = np.array(b)
  if (a==b).all():
    return 1
  else:
    return 0

In [41]:
from sklearn.metrics import accuracy_score
acc=0
false=0

homenumber_true = 0
street_true = 0 
ward_true = 0
district_true = 0
province_true = 0
country_true = 0
postcode_true = 0
ner_true = 0
obj_true = 0
obj_feature_true = 0
pre_true = 0

homenumber_false = 0
street_false = 0
ward_false = 0
district_false = 0
province_false = 0
country_false = 0
postcode_false = 0
ner_false = 0
obj_false = 0
obj_feature_false = 0
pre_false = 0

In [42]:
for i in range(len(y_true)):
  sen_pred = y_pred[i]
  sen_true = y_true[i]
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 1)==1:
    homenumber_true+=1
  else:
    homenumber_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 1)==1:
    homenumber_true+=1
  else:
    homenumber_false+=1

  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 2)==1:
    street_true+=1
  else:
    street_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 3)==1:
    ward_true+=1
  else:
    ward_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 4)==1:
    district_true+=1
  else:
    district_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 5)==1:
    province_true+=1
  else:
    province_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 6)==1:
    country_true+=1
  else:
    country_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 7)==1:
    postcode_true+=1
  else:
    postcode_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 8)==1:
    ner_true+=1
  else:
    ner_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 9)==1:
    obj_true+=1
  else:
    obj_false+=1
  
  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 10)==1:
    obj_feature_true+=1
  else:
    obj_feature_false+=1

  if arr_tag_in_a_tag_sen(sen_pred, sen_true, 11)==1:
    pre_true+=1
  else:
    pre_false+=1

In [43]:
acc = homenumber_true+ street_true+ ward_true+ district_true+ province_true+ country_true+ postcode_true+ ner_true+ obj_true+ obj_feature_true+ pre_true
false = homenumber_false+ street_false+ ward_false+ district_false+ province_false+ country_false+ postcode_false+ ner_false+ obj_false+ obj_feature_false+ pre_false
print(acc)
print(false)
print('accuracy = {}'.format(acc/(acc+false)) )
ti_le_dung = format(acc/(acc+false)*100, '.2f')
print(ti_le_dung)

print("homenumber : {} true, {} false".format(homenumber_true, homenumber_false))
print("street : {} true, {} false".format(street_true, street_false))
print("ward : {} true, {} false".format(ward_true, ward_false))
print("district : {} true, {} false".format(district_true, district_false))
print("province : {} true, {} false".format(province_true, province_false))
print("country : {} true, {} false".format(country_true, country_false))
print("postcode : {} true, {} false".format(postcode_true, postcode_false))
print("ner : {} true, {} false".format(ner_true, ner_false))
print("obj : {} true, {} false".format(obj_true, obj_false))
print("obj_feature : {} true, {} false".format(obj_feature_true, obj_feature_false))
print("pre : {} true, {} false".format(pre_true, pre_false))

5677
5123
accuracy = 0.5256481481481482
52.56
homenumber : 974 true, 826 false
street : 587 true, 313 false
ward : 308 true, 592 false
district : 585 true, 315 false
province : 585 true, 315 false
country : 302 true, 598 false
postcode : 585 true, 315 false
ner : 287 true, 613 false
obj : 589 true, 311 false
obj_feature : 316 true, 584 false
pre : 559 true, 341 false


In [44]:
dict = []
y_true = y_true.reshape(y_true.shape[0]*y_true.shape[1])
y_pred = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1])
for element in y_true:
  if element not in dict:
    dict.append(element)
# print(dict)
dict.sort()
print(dict)

cnf_matrix = confusion_matrix(y_true, y_pred, labels=dict)
for i in range(len(cnf_matrix)):
  for j in range(len(cnf_matrix)):
    cnf_matrix[i][j] = round(cnf_matrix[i][j], 3)
print(cnf_matrix)

arr_to_cal_predict = []
for i in range(len(cnf_matrix)):
  tong= 0
  for j in range(len(cnf_matrix)):
    tong+=cnf_matrix[j][i]
  arr_to_cal_predict.append(tong)
print(arr_to_cal_predict)
# lấy độ predict
predict = []
for i in range(len(cnf_matrix)):
  tong=cnf_matrix[i][i]/arr_to_cal_predict[i]
  predict.append(tong)

print("Độ đo P của từng tag: ")
print(predict)
print(len(predict))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[[26997     0     0     0     0     0     0     0     6     0     0     0
     43]
 [    0   635    18     0     0     0     0     0     0     0     0     6
      0]
 [    0     7  2176    14    14     0     0     0     0     1     0     0
      0]
 [    0     0     0   308     3     0     0     0     0     0     0     0
      0]
 [    0     0     0     0   585    15     0     0     0     0     0     0
      0]
 [    0     0     0     0     0   585     1    14     0     0     0     0
      0]
 [    0     0     0     0     0     0   302     1     0     0     0     0
      0]
 [   14     0     0     0     0     0     0   585     0     0     0     0
      1]
 [    6     0     1     0     0     0     0     4   971     5     1    10
      8]
 [    0     6     5     0     0     0     0     0    16  1120    32     5
      6]
 [    0     0     1     0     0     0     0     0    14    43   659    10
      5]
 [    0     0     6     0     0     0     0 

In [45]:
normalized_confusion_matrix = cnf_matrix/cnf_matrix.sum(axis = 1, keepdims = True)
for i in range(len(normalized_confusion_matrix)):
  for j in range(len(normalized_confusion_matrix)):
    normalized_confusion_matrix[i][j] = round(normalized_confusion_matrix[i][j], 3)
print(normalized_confusion_matrix)

# lấy độ recall
recall = []
for i in range(len(normalized_confusion_matrix)):
  recall.append(normalized_confusion_matrix[i][i])

print("Độ đo R của từng tag: ")
print(recall)
print(len(recall))

[[0.998 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.002]
 [0.    0.964 0.027 0.    0.    0.    0.    0.    0.    0.    0.    0.009
  0.   ]
 [0.    0.003 0.984 0.006 0.006 0.    0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.99  0.01  0.    0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.    0.975 0.025 0.    0.    0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.    0.    0.975 0.002 0.023 0.    0.    0.    0.
  0.   ]
 [0.    0.    0.    0.    0.    0.    0.997 0.003 0.    0.    0.    0.
  0.   ]
 [0.023 0.    0.    0.    0.    0.    0.    0.975 0.    0.    0.    0.
  0.002]
 [0.006 0.    0.001 0.    0.    0.    0.    0.004 0.965 0.005 0.001 0.01
  0.008]
 [0.    0.005 0.004 0.    0.    0.    0.    0.    0.013 0.941 0.027 0.004
  0.005]
 [0.    0.    0.001 0.    0.    0.    0.    0.    0.019 0.059 0.9   0.014
  0.007]
 [0.    0.    0.006 0.    0.    0.    0.    0.    0.006 0.    0.013 0.96
  0.014]
 [0.002 0.    0.001 0.    0

In [46]:
# lấy độ F1
f1 = []
for i in range(len(recall)):
  p = predict[i]
  r = recall[i]
  f1.append(2*p*r/(p+r))

print("Độ đo F1 của từng tag: ")
print(f1)
print(len(f1))

Độ đo F1 của từng tag: 
[0.9985740589471, 0.9719037971789481, 0.9847530464472833, 0.972972972972973, 0.9733777038269551, 0.975, 0.9968498123627069, 0.9717607973421927, 0.9603318575001024, 0.9482592633247737, 0.9157724079363854, 0.9552541513499809, 0.9663260546466452]
13
